In [1]:
# tips: https://towardsdatascience.com/9-tips-for-training-lightning-fast-neural-networks-in-pytorch-8e63a502f565
# https://miguel-data-sc.github.io/2017-11-05-first/
import pandas
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as D
from torch.autograd import Variable
import torch.optim as optim
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import datetime

In [2]:
class MyDataset(D.Dataset):
    def __init__(self, x_tensor, y_tensor):
        self.x = torch.from_numpy(x_tensor)
        self.y = torch.from_numpy(y_tensor)
        
    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.x)

def make_data(dataset, features, batch_size, debug=False):
    datasets = []
    for file in dataset:
        datasets.append(pd.read_csv("../" + file))
    dataset = pd.concat(datasets, axis=0, ignore_index=True)
    # make train and test sets
    target = dataset["affect"]
    tmp_x, test_x, tmp_y, test_y = train_test_split(dataset[features], target, test_size=0.2)
    train_x, val_x, train_y, val_y = train_test_split(tmp_x, tmp_y, test_size=0.2)
    # make data loaders
    train_data = MyDataset(train_x.to_numpy(), train_y.to_numpy())
    val_data = MyDataset(val_x.to_numpy(), val_y.to_numpy())
    test_data = MyDataset(test_x.to_numpy(), test_y.to_numpy())
    train_loader = DataLoader(train_data, batch_size)
    val_loader = DataLoader(val_data, batch_size)
    test_loader = DataLoader(test_data, 1)
    return (train_loader, val_loader, test_loader)

def log(net, summary, epoch, train_loss, val_loss, save_name):
    # print diagram
    if "train" in save_name:
        plt.clf()
        plt.ylabel('loss')
        plt.xlabel('epochs')
        plt.plot(train_loss, label="train_loss")
        plt.plot(val_loss, label="test_loss")
        plt.legend(loc='upper right')
        plt.savefig("../img/" + save_name + str(epoch) + ".png")
    # write in log
    log = open("../logs/" + save_name + ".txt", "a")
    log.write(summary)
    log.close()
    print(summary)
    # save network
    torch.save(net.state_dict(), "../nets/" + save_name + str(epoch) + ".pt")  
    
def update_lr(loss_diff=0, init=False):
	if init: return 0.1
	
	# if loss between x und x: return y
	# if loss between z and z: return w
	
def use_gpu(cuda, net, inputs, targets):
	if cuda:
		net = net.cuda()
		inputs = inputs.cuda()
		targets = targets.cuda()
	return net, inputs, targets

In [3]:
##### Net definition #####

class Lin_Net(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim):
        super(Lin_Net, self).__init__()
        self.lin1 = nn.Linear(input_dim, hidden_dim)
        self.lin2 = nn.Linear(hidden_dim, hidden_dim)
        self.lin3 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = torch.sigmoid(self.lin1(x))
        x = torch.sigmoid(self.lin2(x))
        x = self.lin3(x)
        return x

In [7]:
def train(net, loader, cuda, criterion, optimizer, scheduler):
    net.train()
    for batch, (inputs, targets) in enumerate(loader):
        inputs, targets = inputs.float(), targets.view(targets.shape[0],1).float()
        net, inputs, targets = use_gpu(cuda, net, inputs, targets)
        pred = net(inputs)
        loss = criterion(net(inputs), targets)
        # backpropagate
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # give update
        if (batch % 100) == 0:
            print("... train batch {}/{}, lr:{}".format(batch, len(loader), lr))
    return net, loss.item()
    
def val(net, loader, cuda, criterion):
    net.eval()
    for batch, (inputs, targets) in enumerate(loader):
        inputs, targets = inputs.float(), targets.float().view(targets.shape[0],1)
        net, inputs, targets = use_gpu(cuda, net, inputs, targets)
        loss = criterion(net(inputs), targets)
        # give update
        if (batch % 100) == 0:
            print("...... val batch {}/{}".format(batch, len(loader)))
    return net, loss.item()

def test(net, loader, cuda, criterion):
    net.eval()
    loss_sum, correct, correct2 = 0, 0, 0
    confusion = []
    for batch, (inputs, targets) in enumerate(loader):
        inputs, targets = inputs.float(), targets.float()
        net, inputs, targets = use_gpu(cuda, net, inputs, targets)
        pred = net(inputs)
        pred_class = round(pred.item())
        loss_sum += criterion(pred, targets).item()
        confusion.append([targets.item(), pred_class])
        # give update
        if (batch % 100 == 0):
            print("...... test batch {}/{}".format(batch, len(loader)))
    correct = sum((pair.count(pair[0]) == len(pair)) for pair in confusion)
    # return avg loss, avg correct, confusion
    return (loss_sum / len(test_loader)), (correct / len(test_loader)), confusion


def fit(net, data, cuda, criterion, epochs, log_every, save_name):
    open("../logs/" + save_name + "_train.txt", "w").close()
    open("../logs/" + save_name + "_test.txt", "w").close()
    optimizer = optim.SGD(net.parameters(), 0.1, momentum=0.5)
    #scheduler = optim.lr_scheduler.LambdaLR(optimizer, len(data[0]))
    scheduler = None
    running_train_loss, running_val_loss = [], []
    
    for epoch in range(epochs):
        #scheduler.step()
        net, train_loss = train(net, data[0], cuda, criterion, optimizer, scheduler)
        running_train_loss.append(train_loss)
        net, val_loss =  val(net, data[1], cuda, criterion)
        running_val_loss.append(val_loss)
        
        if (epoch % log_every[0]) == 0:
            summary = "{}: epoch {}/{}:\t train_loss: {}\t val_loss: {}".format(datetime.datetime.now(), epoch, epochs, train_loss, val_loss)
            log(net, summary, epoch, running_train_loss, running_val_loss, save_name + "_train")
    
    avg_loss, avg_correct, confusion = test(net, data[2], cuda, criterion)
    summary = "{}: avg_loss: {} avg_correct: {}\n\n{}".format(datetime.datetime.now(), avg_loss, avg_correct, confusion)
    log(net, summary, epoch, running_train_loss, running_val_loss, save_name + "_test")
            

In [5]:
# create variables 
print("creating variables")
emotion_dataset = ["emotion_classification_1_clean.csv", "emotion_classification_2_clean.csv", "emotion_classification_3_clean.csv", "emotion_classification_4_clean.csv", "emotion_classification_5_clean.csv", "emotion_classification_6_clean.csv", "emotion_classification_7_clean.csv", "emotion_classification_8_clean.csv"]
tweet_dataset = ["crowdflower_clean.csv", "emoint_clean.csv", "tec_clean.csv"]
full_features = ["word_count", "upper_word_count", "ent_word_count", "h_count", "s_count", "a_count", "f_count", "cons_punct_count"]
nolex_features = ["word_count", "upper_word_count", "ent_word_count", "cons_punct_count"]
lex_features = ["h_count", "s_count", "a_count", "f_count"]
log_every = [50, 2500] # epochs and batches
epochs = 50
batch_size = 64
cuda = torch.cuda.is_available()
criterion = nn.MSELoss()
print("... done")

creating variables
... done


In [8]:
print("-------- net_lin_emotion_full")
net = Lin_Net(8, 1, 64)
data = make_data(tweet_dataset, full_features, batch_size)
fit(net, data, cuda, criterion, epochs, log_every, "lin_emotion_full")






-------- net_lin_emotion_full


NameError: name 'lr' is not defined

In [ ]:


print("-------- net_lin_emotion_full")
net_full = Lin_Net(8, 1, 64)
train_loader_emotion_full, test_loader_emotion_full = make_data(emotion_dataset, "full", batch_size)
train(train_loader_emotion_full, net_full, epochs, print_every, "mse_emotion_full", cuda)
#test(test_loader_emotion_full, net_full, print_every_epoch0, "mse_emotion_full")

print("-------- net_lin_emotion_nolex")
net_half = Lin_Net(4, 1, 64)
train_loader_emotion_nolex, test_loader_emotion_nolex = make_data(emotion_dataset, "nolex", batch_size)
train(train_loader_emotion_nolex, net_half, epochs, print_every, "mse_emotion_nolex", cuda)
#test(test_loader_emotion_nolex, net_half, print_every_epoch0, "mse_emotion_nolex")

print("-------- net_lin_emotion_lex")
net_half = Lin_Net(4, 1, 64)
train_loader_emotion_lex, test_loader_emotion_lex = make_data(emotion_dataset, "lex", batch_size)
train(train_loader_emotion_lex, net_half, epochs, print_every, "mse_emotion_lex", cuda)
#test(test_loader_emotion_lex, net_half, print_every_epoch0, "mse_emotion_lex")

print("-------- net_lin_tweet_full")
net_full = Lin_Net(8, 1, 64)
train_loader_tweet_full, test_loader_tweet_full = make_data(tweet_dataset, "full", batch_size)
train(train_loader_tweet_full, net_full, epochs, print_every, "mse_tweet_full", cuda)
#test(test_loader_tweet_full, net_full, print_every_epoch0, "mse_tweet_full")

print("-------- net_lin_tweet_nolex")
net_half = Lin_Net(4, 1, 64)
train_loader_tweet_nolex, test_loader_tweet_nolex = make_data(tweet_dataset, "nolex", batch_size)
train(train_loader_tweet_nolex, net_half, epochs, print_every, "mse_tweet_nolex", cuda)
#test(test_loader_tweet_nolex, net_half, print_every_epoch0, "mse_tweet_nolex")

print("-------- net_lin_tweet_lex")
net_half = Lin_Net(4, 1, 64)
train_loader_tweet_lex, test_loader_tweet_lex = make_data(tweet_dataset, "lex", batch_size)
train(train_loader_tweet_lex, net_half, epochs, print_every, "mse_tweet_lex", cuda)
#test(test_loader_tweet_lex, net_half, print_every_epoch0, "mse_tweet_lex")
print("... done")